In [1]:
!which python

/projappl/project_2013104/pengyan1/venv/bin/python


In [2]:
import sys
import os

folder_path = os.path.abspath("/projappl/project_2013104/pengyan1/venv/lib/python3.10/site-packages")
if folder_path not in sys.path:
    sys.path.append(folder_path)

sys.path

['/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python310.zip',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/lib-dynload',
 '',
 '/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages',
 '/projappl/project_2013104/pengyan1/venv/lib/python3.10/site-packages']

In [1]:
import torch
import torch.nn.functional as F
from torchmetrics import JaccardIndex
import torch_geometric.transforms as T
from torch_geometric.datasets import S3DIS
from torch_geometric.loader import DataLoader
from torch_geometric.typing import WITH_TORCH_CLUSTER
from pyg_pointnet2 import PyGPointNet2

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

In [3]:
print(torch.cuda.is_available())

True


In [2]:
test_area = 6  
#path = "/scratch/project_2013104/datasets/s3dis/" # CSC
path = r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\AI\scripts\datasets\s3dis" # Local

# transform and pre_transform
transform = T.Compose([
    T.RandomJitter(0.01),
    T.RandomRotate(15, axis=0),
    T.RandomRotate(15, axis=1),
    T.RandomRotate(15, axis=2)
])
pre_transform = T.NormalizeScale()

In [4]:
train_dataset = S3DIS(root=path, test_area=test_area, train=True, force_reload=True, 
                      transform=transform,pre_transform=pre_transform)
test_dataset = S3DIS(root=path, test_area=test_area, train=False, force_reload=True,
                     pre_transform=pre_transform)

C:\Users\yanpe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch_geometric\data\dataset.py:240: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(
Processing...
Done!
Processing...
Done!


In [5]:
batch_size=64
num_workers=10

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PyGPointNet2(train_dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
def train():
    model.train()

    total_loss = correct_nodes = total_nodes = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct_nodes += out.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        if (i + 1) % 10 == 0:
            print(f'[{i+1}/{len(train_loader)}] Loss: {total_loss / 10:.4f} '
                  f'Train Acc: {correct_nodes / total_nodes:.4f}')
            total_loss = correct_nodes = total_nodes = 0



In [8]:
@torch.no_grad()
def test(loader):
    model.eval()
    jaccard = JaccardIndex(num_classes=loader.dataset.num_classes, task="multiclass").to(device)
    
    for data in loader:
        data = data.to(device)
        outs = model(data)
        preds = outs.argmax(dim=-1)
        jaccard.update(preds, data.y)
    
    return jaccard.compute().item()


In [9]:
import time
for epoch in range(1, 46):
    # Track epoch start time
    start_time = time.perf_counter()
    train()
    iou = test(test_loader)
    # Calculate epoch duration
    epoch_time = time.perf_counter() - start_time
    
    # Print results with time
    print(f'Epoch: {epoch:02d}, Test IoU: {iou:.4f}, Time: {epoch_time:.2f}s')

[10/318] Loss: 2.1916 Train Acc: 0.3953
[20/318] Loss: 1.3879 Train Acc: 0.5984
[30/318] Loss: 1.1889 Train Acc: 0.6488
[40/318] Loss: 1.1222 Train Acc: 0.6493
[50/318] Loss: 0.9658 Train Acc: 0.6909
[60/318] Loss: 0.9449 Train Acc: 0.7036
[70/318] Loss: 0.9375 Train Acc: 0.6996
[80/318] Loss: 0.8623 Train Acc: 0.7208
[90/318] Loss: 0.8488 Train Acc: 0.7245
[100/318] Loss: 0.8173 Train Acc: 0.7293
[110/318] Loss: 0.7731 Train Acc: 0.7317
[120/318] Loss: 0.8354 Train Acc: 0.7307
[130/318] Loss: 0.7494 Train Acc: 0.7546
[140/318] Loss: 0.8204 Train Acc: 0.7286
[150/318] Loss: 0.8007 Train Acc: 0.7423
[160/318] Loss: 0.7404 Train Acc: 0.7583
[170/318] Loss: 0.7580 Train Acc: 0.7551
[180/318] Loss: 0.7084 Train Acc: 0.7686
[190/318] Loss: 0.7759 Train Acc: 0.7533
[200/318] Loss: 0.6915 Train Acc: 0.7869
[210/318] Loss: 0.7340 Train Acc: 0.7671
[220/318] Loss: 0.7280 Train Acc: 0.7647
[230/318] Loss: 0.7033 Train Acc: 0.7779
[240/318] Loss: 0.6703 Train Acc: 0.7882
[250/318] Loss: 0.6806 Tr

KeyboardInterrupt: 

In [30]:

checkpoint_path = "/scratch/project_2013104/checkpoints/pointnet2_s3dis_transform_seg_x6_45_checkpoint.pth"

# Save model, optimizer state, and any other info needed
torch.save({
    'epoch': 45,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    #'loss': loss,
    #'test_accuracy': test_acc
}, checkpoint_path)

print("Checkpoint saved successfully!")

Checkpoint saved successfully!
